In [5]:
import pickle

In [6]:
PATH='/home/kirana/Documents/phd/experiment1_new/DNNREC_ML1M'
DATAPATH='/home/kirana/Documents/final_dissertation_final/experiments/datasets/ml-1m'

In [7]:
MYMEDIALITEPATH='/home/kirana/MyMediaLite/bin'

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import sklearn as sklearn
import datetime as dt 
import seaborn as sns
import torch as torch

In [9]:
from sklearn import metrics

In [10]:
[dftrain,dfvalid]=pickle.load(open(f'{DATAPATH}/df_side_cold.pkl','rb'))

In [11]:
dftrain['rating'].describe()

count    895738.000000
mean          3.597991
std           1.112852
min           1.000000
25%           3.000000
50%           4.000000
75%           4.000000
max           5.000000
Name: rating, dtype: float64

In [12]:
dfinp_train=dftrain[['userId','itemId','rating']]
dfinp_test=dfvalid[['userId','itemId','rating']]
dfinp_train.to_csv(f'{PATH}/inp_train_ml1m.csv',index=False,header=False,sep=",")
dfinp_test.to_csv(f'{PATH}/inp_test_ml1m.csv',index=False,header=False,sep=",")

In [13]:
TRAINFILEPATH=f'{PATH}/inp_train_ml1m.csv'
TESTFILEPATH=f'{PATH}/inp_test_ml1m.csv'

In [14]:
dfinp_train.head()

,userId,itemId,rating
0,670,312,3
1,670,2091,2
2,670,1642,2
3,670,2628,1
4,670,473,1


### Global Average

In [15]:
cmd=f'{MYMEDIALITEPATH}/rating_prediction --training-file={TRAINFILEPATH} --recommender=GlobalAverage \
--test-file={TESTFILEPATH} --prediction-file {DATAPATH}/prediction_globalavg.txt'

In [16]:
!command {cmd}

=loading_time 1.1
memory 16
ratings range: [1, 5]
training data: 5973 users, 3685 items, 895738 ratings, sparsity 95.93041
test data:     5432 users, 3448 items, 104471 ratings, sparsity 99.44221
GlobalAverage  training_time 00:00:00.0062130 RMSE 1.15402 MAE 0.9539394 CBD 0.2918496 new users: RMSE 1.0944 MAE 0.91741 CBD 0.27826 new items: RMSE 1.09026 MAE 0.9683 CBD 0.31621  new users and items: RMSE 1.07778 MAE 1 CBD 0.25439 testing_time 00:00:00.0428420
prediction_time 00:00:00.1001310

memory 16
>

In [17]:
predictions_globalavg=pd.read_csv (f'{DATAPATH}/prediction_globalavg.txt',sep='\t',header=None).iloc[:,2]

In [18]:
dfvalid['pred_globalavg']=predictions_globalavg

In [19]:
def score_by_group (dfvalid,predVar,groupVar='cold_start_group',method='global_avg'):
    scores=[]
    mae_scores=[]
    r2_scores=[]
    groups=dfvalid[groupVar].value_counts().index.tolist()
    for group in groups:
        df=dfvalid.loc[dfvalid[groupVar]==group]
        actual=df['rating']
        score=metrics.mean_squared_error(df['rating'],df[predVar])
        mae_score=metrics.mean_absolute_error(df['rating'],df[predVar])
        r2_score=metrics.r2_score(df['rating'],df[predVar])
        scores.append(score)
        mae_scores.append(mae_score)
        r2_scores.append(r2_score)
    return pd.DataFrame({'method':method, 'mse':scores, 'rmse':np.sqrt(scores), 'mae':mae_scores, 'r2_score':r2_scores, 'Cold Start Group':groups})

In [20]:
result_globalavg=pd.concat([score_by_group(dfvalid,'pred_globalavg'),score_by_group(dfvalid,'pred_globalavg','cold_start_bucket'),\
score_by_group(dfvalid,'pred_globalavg','flag_train')],axis=0)

In [21]:
result_globalavg

,method,mse,rmse,mae,r2_score,Cold Start Group
0,global_avg,1.344811,1.159660,0.957483,-0.023810,No-cold-start
1,global_avg,1.197711,1.094400,0.917391,-0.001733,User-Cold-Start
2,global_avg,1.190927,1.091296,0.965662,-0.429112,Item-Cold_start
3,global_avg,1.161611,1.077781,1.000000,-0.161611,User-Item-cold-start
0,global_avg,1.344811,1.159660,0.957483,-0.023810,0
1,global_avg,1.197686,1.094388,0.917534,-0.001614,1
0,global_avg,1.331762,1.154020,0.953939,-0.018923,0


In [22]:
dfvalid['flag_train'].value_counts()

0    104471
Name: flag_train, dtype: int64

### User Average

In [23]:
cmd=f'{MYMEDIALITEPATH}/rating_prediction --training-file={TRAINFILEPATH} --recommender=UserAverage \
--test-file={TESTFILEPATH} --prediction-file {DATAPATH}/prediction_useravg.txt'

In [24]:
!command {cmd}

=loading_time 1.11
memory 16
ratings range: [1, 5]
training data: 5973 users, 3685 items, 895738 ratings, sparsity 95.93041
test data:     5432 users, 3448 items, 104471 ratings, sparsity 99.44221
UserAverage  training_time 00:00:00.0268390 RMSE 1.083289 MAE 0.8627017 CBD 0.2828474 new users: RMSE 1.0944 MAE 0.91741 CBD 0.27826 new items: RMSE 0.98704 MAE 0.82639 CBD 0.30524  new users and items: RMSE 1.07778 MAE 1 CBD 0.25439 testing_time 00:00:00.0457440
prediction_time 00:00:00.0990450

memory 16
>

In [25]:
predictions_useravg=pd.read_csv (f'{DATAPATH}/prediction_useravg.txt',sep='\t',header=None).iloc[:,2]

In [26]:
dfvalid['pred_useravg']=predictions_useravg

In [27]:
result_useravg=pd.concat([score_by_group(dfvalid,'pred_useravg','cold_start_group','useravg'),score_by_group(dfvalid,'pred_useravg','cold_start_bucket','useravg'),\
score_by_group(dfvalid,'pred_useravg','flag_train','useravg')],axis=0)
result_useravg

,method,mse,rmse,mae,r2_score,Cold Start Group
0,useravg,1.171221,1.082230,0.857404,0.108345,No-cold-start
1,useravg,1.197711,1.094400,0.917391,-0.001733,User-Cold-Start
2,useravg,0.958643,0.979103,0.811926,-0.150372,Item-Cold_start
3,useravg,1.161611,1.077781,1.000000,-0.161611,User-Item-cold-start
0,useravg,1.171221,1.082230,0.857404,0.108345,0
1,useravg,1.197084,1.094113,0.917136,-0.001110,1
0,useravg,1.173515,1.083289,0.862702,0.102151,0


### Item Average

In [28]:
cmd=f'{MYMEDIALITEPATH}/rating_prediction --training-file={TRAINFILEPATH} --recommender=ItemAverage \
--test-file={TESTFILEPATH} --prediction-file {DATAPATH}/prediction_itemavg.txt'

In [29]:
!command {cmd}

=loading_time 1.16
memory 16
ratings range: [1, 5]
training data: 5973 users, 3685 items, 895738 ratings, sparsity 95.93041
test data:     5432 users, 3448 items, 104471 ratings, sparsity 99.44221
ItemAverage  training_time 00:00:00.0263350 RMSE 0.9923189 MAE 0.7899396 CBD 0.2717245 new users: RMSE 0.98425 MAE 0.78976 CBD 0.2643 new items: RMSE 1.09026 MAE 0.9683 CBD 0.31621  new users and items: RMSE 1.07778 MAE 1 CBD 0.25439 testing_time 00:00:00.0455190
prediction_time 00:00:00.1090690

memory 16
>

In [30]:
predictions_itemavg=pd.read_csv (f'{DATAPATH}/prediction_itemavg.txt',sep='\t',header=None).iloc[:,2]

In [31]:
dfvalid['pred_itemavg']=predictions_itemavg

In [32]:
result_itemavg=pd.concat([score_by_group(dfvalid,'pred_itemavg','cold_start_group','itemavg'),score_by_group(dfvalid,'pred_itemavg','cold_start_bucket','itemavg'),\
score_by_group(dfvalid,'pred_itemavg','flag_train','itemavg')],axis=0)
result_itemavg

,method,mse,rmse,mae,r2_score,Cold Start Group
0,itemavg,0.986194,0.993073,0.789912,0.249207,No-cold-start
1,itemavg,0.968701,0.984226,0.789719,0.189805,User-Cold-Start
2,itemavg,1.190927,1.091296,0.965662,-0.429112,Item-Cold_start
3,itemavg,1.161611,1.077781,1.000000,-0.161611,User-Item-cold-start
0,itemavg,0.986194,0.993073,0.789912,0.249207,0
1,itemavg,0.969318,0.984540,0.790220,0.189368,1
0,itemavg,0.984697,0.992319,0.789940,0.246615,0


## Neighborhood Methods

### Item KNN Cosine

In [33]:
cmd=f'{MYMEDIALITEPATH}/rating_prediction --training-file={TRAINFILEPATH} --recommender=ItemKNN \
--test-file={TESTFILEPATH} --prediction-file {DATAPATH}/prediction_itemknn_cosine.txt \
--recommender-options k=80  --recommender-options reg_u=25 \
--recommender-options reg_i=10'

In [34]:
!command {cmd}

=loading_time 1.2
memory 41
ratings range: [1, 5]
training data: 5973 users, 3685 items, 895738 ratings, sparsity 95.93041
test data:     5432 users, 3448 items, 104471 ratings, sparsity 99.44221
ItemKNN k=80 correlation=BinaryCosine weighted_binary=False alpha=0; baseline predictor: reg_u=25 reg_i=10 num_iter=10  training_time 00:00:46.1416550 RMSE 0.9093671 MAE 0.7134065 CBD 0.2619095 new users: RMSE 0.984 MAE 0.78711 CBD 0.26432 new items: RMSE 0.9785 MAE 0.85462 CBD 0.30359  new users and items: RMSE 1.07778 MAE 1 CBD 0.25439 testing_time 00:14:19.3961410
prediction_time 00:14:19.1880110

memory 118
>

In [35]:
predictions_itemknn_cosine=pd.read_csv (f'{DATAPATH}/prediction_itemknn_cosine.txt',sep='\t',header=None).iloc[:,2]
dfvalid['pred_itemknn_cosine']=predictions_itemknn_cosine

In [36]:
result_itemknncosine=pd.concat([score_by_group(dfvalid,'pred_itemknn_cosine','cold_start_group','itemknncosine'),score_by_group(dfvalid,'pred_itemknn_cosine','cold_start_bucket','itemknncosine'),\
score_by_group(dfvalid,'pred_itemknn_cosine','flag_train','itemknncosine')],axis=0)
result_itemknncosine

,method,mse,rmse,mae,r2_score,Cold Start Group
0,itemknncosine,0.813202,0.901777,0.706219,0.380906,No-cold-start
1,itemknncosine,0.968217,0.983980,0.787066,0.190210,User-Cold-Start
2,itemknncosine,0.940441,0.969763,0.842502,-0.128529,Item-Cold_start
3,itemknncosine,1.161611,1.077781,1.000000,-0.161611,User-Item-cold-start
0,itemknncosine,0.813202,0.901777,0.706219,0.380906,0
1,itemknncosine,0.968187,0.983965,0.787255,0.190315,1
0,itemknncosine,0.826949,0.909367,0.713406,0.367307,0


### Item KNN Pearson

In [37]:
cmd=f'{MYMEDIALITEPATH}/rating_prediction --training-file={TRAINFILEPATH} --recommender=ItemKNN \
--test-file={TESTFILEPATH} --prediction-file {DATAPATH}/prediction_itemknn_pearson.txt \
--recommender-options k=80  --recommender-options reg_u=25 \
--recommender-options reg_i=10 --recommender-options correlation=Pearson'

In [38]:
!command {cmd}

=loading_time 1.22
memory 41
ratings range: [1, 5]
training data: 5973 users, 3685 items, 895738 ratings, sparsity 95.93041
test data:     5432 users, 3448 items, 104471 ratings, sparsity 99.44221
ItemKNN k=80 correlation=Pearson weighted_binary=False alpha=0; baseline predictor: reg_u=25 reg_i=10 num_iter=10  training_time 00:00:19.3995810 RMSE 0.9041163 MAE 0.7073354 CBD 0.2613333 new users: RMSE 0.984 MAE 0.78711 CBD 0.26432 new items: RMSE 0.9785 MAE 0.85462 CBD 0.30359  new users and items: RMSE 1.07778 MAE 1 CBD 0.25439 testing_time 00:07:25.7768800
prediction_time 00:07:25.9212430

memory 118
>

In [39]:
predictions_itemknn_pearson=pd.read_csv (f'{DATAPATH}/prediction_itemknn_pearson.txt',sep='\t',header=None).iloc[:,2]
dfvalid['pred_itemknn_pearson']=predictions_itemknn_pearson

In [40]:
result_itemknnpearson=pd.concat([score_by_group(dfvalid,'pred_itemknn_pearson','cold_start_group','itemknnpearson'),score_by_group(dfvalid,'pred_itemknn_pearson','cold_start_bucket','itemknnpearson'),\
score_by_group(dfvalid,'pred_itemknn_pearson','flag_train','itemknnpearson')],axis=0)
result_itemknnpearson

,method,mse,rmse,mae,r2_score,Cold Start Group
0,itemknnpearson,0.802753,0.895965,0.699557,0.388861,No-cold-start
1,itemknnpearson,0.968217,0.983980,0.787066,0.190210,User-Cold-Start
2,itemknnpearson,0.940441,0.969763,0.842502,-0.128529,Item-Cold_start
3,itemknnpearson,1.161611,1.077781,1.000000,-0.161611,User-Item-cold-start
0,itemknnpearson,0.802753,0.895965,0.699557,0.388861,0
1,itemknnpearson,0.968187,0.983965,0.787255,0.190315,1
0,itemknnpearson,0.817426,0.904116,0.707335,0.374592,0


### User KNN Pearson

In [45]:
cmd=f'{MYMEDIALITEPATH}/rating_prediction --training-file={TRAINFILEPATH} --recommender=UserKNN \
--test-file={TESTFILEPATH} --prediction-file {DATAPATH}/prediction_userknn_pearson.txt \
--recommender-options k=80  --recommender-options reg_u=25 \
--recommender-options reg_i=10 --recommender-options correlation=Pearson'

In [46]:
!command {cmd}

=loading_time 1.14
memory 38
ratings range: [1, 5]
training data: 5973 users, 3685 items, 895738 ratings, sparsity 95.93041
test data:     5432 users, 3448 items, 104471 ratings, sparsity 99.44221
UserKNN k=80 correlation=Pearson weighted_binary=False alpha=0; baseline predictor: reg_u=25 reg_i=10 num_iter=10  training_time 00:00:55.6236500 RMSE 0.9152783 MAE 0.7187243 CBD 0.2629423 new users: RMSE 0.984 MAE 0.78711 CBD 0.26432 new items: RMSE 0.9785 MAE 0.85462 CBD 0.30359  new users and items: RMSE 1.07778 MAE 1 CBD 0.25439 testing_time 00:19:11.7622810
prediction_time 00:19:57.6011710

memory 249
>

In [47]:
predictions_userknn_pearson=pd.read_csv (f'{DATAPATH}/prediction_userknn_pearson.txt',sep='\t',header=None).iloc[:,2]
dfvalid['pred_userknn_pearson']=predictions_userknn_pearson

In [48]:
result_userknnpearson=pd.concat([score_by_group(dfvalid,'pred_userknn_pearson','cold_start_group','userknnpearson'),score_by_group(dfvalid,'pred_userknn_pearson','cold_start_bucket','userknnpearson'),\
score_by_group(dfvalid,'pred_userknn_pearson','flag_train','userknnpearson')],axis=0)
result_userknnpearson

,method,mse,rmse,mae,r2_score,Cold Start Group
0,userknnpearson,0.825038,0.908316,0.712054,0.371896,No-cold-start
1,userknnpearson,0.968217,0.983980,0.787066,0.190210,User-Cold-Start
2,userknnpearson,0.940441,0.969763,0.842502,-0.128529,Item-Cold_start
3,userknnpearson,1.161611,1.077781,1.000000,-0.161611,User-Item-cold-start
0,userknnpearson,0.825038,0.908316,0.712054,0.371896,0
1,userknnpearson,0.968187,0.983965,0.787255,0.190315,1
0,userknnpearson,0.837734,0.915278,0.718724,0.359055,0


### User KNN Cosine

In [65]:
cmd=f'{MYMEDIALITEPATH}/rating_prediction --training-file={TRAINFILEPATH} --recommender=UserKNN \
--test-file={TESTFILEPATH} --prediction-file {DATAPATH}/prediction_userknn_cosine.txt \
--recommender-options k=80  --recommender-options reg_u=25 \
--recommender-options reg_i=10'

In [66]:
!command {cmd}

=loading_time 1.18
memory 38
ratings range: [1, 5]
training data: 5973 users, 3685 items, 895738 ratings, sparsity 95.93041
test data:     5432 users, 3448 items, 104471 ratings, sparsity 99.44221
UserKNN k=80 correlation=BinaryCosine weighted_binary=False alpha=0; baseline predictor: reg_u=25 reg_i=10 num_iter=10  training_time 00:01:45.7271230 RMSE 0.9225795 MAE 0.7242093 CBD 0.2636982 new users: RMSE 0.984 MAE 0.78711 CBD 0.26432 new items: RMSE 0.9785 MAE 0.85462 CBD 0.30359  new users and items: RMSE 1.07778 MAE 1 CBD 0.25439 testing_time 00:28:37.2978810
prediction_time 00:28:46.1241780

memory 249
>

In [67]:
predictions_userknn_cosine=pd.read_csv (f'{DATAPATH}/prediction_userknn_cosine.txt',sep='\t',header=None).iloc[:,2]
dfvalid['pred_userknn_cosine']=predictions_userknn_cosine

In [68]:
result_userknncosine=pd.concat([score_by_group(dfvalid,'pred_userknn_cosine','cold_start_group','userknncosine'),score_by_group(dfvalid,'pred_userknn_cosine','cold_start_bucket','userknncosine'),\
score_by_group(dfvalid,'pred_userknn_cosine','flag_train','userknncosine')],axis=0)
result_userknncosine

,method,mse,rmse,mae,r2_score,Cold Start Group
0,userknncosine,0.839762,0.916385,0.718073,0.360686,No-cold-start
1,userknncosine,0.968217,0.983980,0.787066,0.190210,User-Cold-Start
2,userknncosine,0.940441,0.969763,0.842502,-0.128529,Item-Cold_start
3,userknncosine,1.161611,1.077781,1.000000,-0.161611,User-Item-cold-start
0,userknncosine,0.839762,0.916385,0.718073,0.360686,0
1,userknncosine,0.968187,0.983965,0.787255,0.190315,1
0,userknncosine,0.851153,0.922579,0.724209,0.348788,0


## Matrix Factorization

### SVD

In [54]:
cmd=f'{MYMEDIALITEPATH}/rating_prediction --training-file={TRAINFILEPATH} --recommender=MatrixFactorization \
--test-file={TESTFILEPATH} --prediction-file {DATAPATH}/prediction_svd.txt \
--recommender-options num_factors=10  --recommender-options regularization=0.05 \
--recommender-options learn_rate=0.005 --recommender-options num_iter=75'

In [55]:
!command {cmd}

=loading_time 1.13
memory 16
ratings range: [1, 5]
training data: 5973 users, 3685 items, 895738 ratings, sparsity 95.93041
test data:     5432 users, 3448 items, 104471 ratings, sparsity 99.44221
MatrixFactorization num_factors=10 regularization=0.05 learn_rate=0.005 learn_rate_decay=1 num_iter=75  training_time 00:00:18.2684940 RMSE 0.8976225 MAE 0.7074416 CBD 0.2604465 new users: RMSE 1.0944 MAE 0.91741 CBD 0.27826 new items: RMSE 1.09026 MAE 0.9683 CBD 0.31621  new users and items: RMSE 1.07778 MAE 1 CBD 0.25439 testing_time 00:00:00.0273180
prediction_time 00:00:00.1134340

memory 20
>

In [56]:
predictions_svd=pd.read_csv (f'{DATAPATH}/prediction_svd.txt',sep='\t',header=None).iloc[:,2]
dfvalid['pred_svd']=predictions_svd

In [57]:
result_svd=pd.concat([score_by_group(dfvalid,'pred_svd','cold_start_group','svd'),score_by_group(dfvalid,'pred_svd','cold_start_bucket','svd'),\
score_by_group(dfvalid,'pred_svd','flag_train','svd')],axis=0)
result_svd

,method,mse,rmse,mae,r2_score,Cold Start Group
0,svd,0.767578,0.876115,0.686994,0.415640,No-cold-start
1,svd,1.197711,1.094400,0.917391,-0.001733,User-Cold-Start
2,svd,1.190927,1.091296,0.965662,-0.429112,Item-Cold_start
3,svd,1.161611,1.077781,1.000000,-0.161611,User-Item-cold-start
0,svd,0.767578,0.876115,0.686994,0.415640,0
1,svd,1.197686,1.094388,0.917534,-0.001614,1
0,svd,0.805726,0.897623,0.707442,0.383544,0


### SVD Plus Plus

In [58]:
cmd=f'{MYMEDIALITEPATH}/rating_prediction --training-file={TRAINFILEPATH} --recommender=SVDPlusPlus \
--test-file={TESTFILEPATH} --prediction-file {DATAPATH}/prediction_svdplusplus.txt \
--recommender-options num_factors=20  --recommender-options regularization=0.05 \
--recommender-options learn_rate=0.005 --recommender-options num_iter=80'



In [59]:
cmd

'/home/kirana/MyMediaLite/bin/rating_prediction --training-file=/home/kirana/Documents/phd/experiment1_new/DNNREC_ML1M/inp_train_ml1m.csv --recommender=SVDPlusPlus --test-file=/home/kirana/Documents/phd/experiment1_new/DNNREC_ML1M/inp_test_ml1m.csv --prediction-file /home/kirana/Documents/final_dissertation_final/experiments/datasets/ml-1m/prediction_svdplusplus.txt --recommender-options num_factors=20  --recommender-options regularization=0.05 --recommender-options learn_rate=0.005 --recommender-options num_iter=80'

In [60]:
!command {cmd}

=loading_time 1.18
memory 16
ratings range: [1, 5]
training data: 5973 users, 3685 items, 895738 ratings, sparsity 95.93041
test data:     5432 users, 3448 items, 104471 ratings, sparsity 99.44221
SVDPlusPlus num_factors=20 regularization=0.05 bias_reg=0.33 frequency_regularization=False learn_rate=0.005 bias_learn_rate=0.7 learn_rate_decay=1 num_iter=80  training_time 01:36:15.6995110 RMSE 0.8824787 MAE 0.6908272 CBD 0.2590143 new users: RMSE 0.9827 MAE 0.78941 CBD 0.26411 new items: RMSE 1.00855 MAE 0.88684 CBD 0.30689  new users and items: RMSE 1.07778 MAE 1 CBD 0.25439 testing_time 00:00:00.0616770
prediction_time 00:00:00.1129690

memory 33
>

In [61]:
predictions_svdplusplus=pd.read_csv (f'{DATAPATH}/prediction_svdplusplus.txt',sep='\t',header=None).iloc[:,2]
dfvalid['pred_svdplusplus']=predictions_svdplusplus

result_svdplusplus=pd.concat([score_by_group(dfvalid,'pred_svdplusplus','cold_start_group','svdplusplus'),score_by_group(dfvalid,'pred_svdplusplus','cold_start_bucket','svdplusplus'),\
score_by_group(dfvalid,'pred_svdplusplus','flag_train','svdplusplus')],axis=0)
result_svdplusplus

,method,mse,rmse,mae,r2_score,Cold Start Group
0,svdplusplus,0.760565,0.872104,0.681210,0.420979,No-cold-start
1,svdplusplus,0.965663,0.982682,0.789363,0.192346,User-Cold-Start
2,svdplusplus,1.005146,1.002569,0.877412,-0.206175,Item-Cold_start
3,svdplusplus,1.161611,1.077781,1.000000,-0.161611,User-Item-cold-start
0,svdplusplus,0.760565,0.872104,0.681210,0.420979,0
1,svdplusplus,0.965808,0.982755,0.789637,0.192304,1
0,svdplusplus,0.778769,0.882479,0.690827,0.404169,0


### BiasedMatrixFactorization

In [62]:
cmd=f'{MYMEDIALITEPATH}/rating_prediction --training-file={TRAINFILEPATH} --recommender=BiasedMatrixFactorization \
--test-file={TESTFILEPATH} --prediction-file {DATAPATH}/prediction_biasedmatrixfactorization.txt \
--recommender-options num_factors=40   --recommender-options bias_reg=0.001 \
--recommender-options learn_rate=0.07 --recommender-options num_iter=110 \
--recommender-options bold_driver=True \
--recommender-options regularization=0.06'

In [63]:
!command {cmd}

=loading_time 1.09
memory 16
ratings range: [1, 5]
training data: 5973 users, 3685 items, 895738 ratings, sparsity 95.93041
test data:     5432 users, 3448 items, 104471 ratings, sparsity 99.44221
BiasedMatrixFactorization num_factors=40 bias_reg=0.001 reg_u=0.06 reg_i=0.06 frequency_regularization=False learn_rate=0.07 bias_learn_rate=1 learn_rate_decay=1 num_iter=110 bold_driver=True loss=RMSE max_threads=1 naive_parallelization=False objective 745811.25 learn_rate 0.0735 
objective 727263.25 learn_rate 0.077175 
objective 718768.6875 learn_rate 0.08103374 
objective 715622.4375 learn_rate 0.08508543 
objective 715790.875 learn_rate 0.04254271 
objective 678467 learn_rate 0.04466985 
objective 675853.6875 learn_rate 0.04690334 
objective 674908.0625 learn_rate 0.0492485 
objective 674838.1875 learn_rate 0.05171093 
objective 675444.75 learn_rate 0.02585546 
objective 655313.75 learn_rate 0.02714824 
objective 653830.625 learn_rate 0.02850565 
objective 653363.5625 learn_rate 0.02993

In [64]:
predictions_biasedmatrixfactorization=pd.read_csv (f'{DATAPATH}/prediction_biasedmatrixfactorization.txt',sep='\t',header=None).iloc[:,2]
dfvalid['pred_biasedmatrixfactorization']=predictions_biasedmatrixfactorization

result_biasedmatrixfactorization=pd.concat([score_by_group(dfvalid,'pred_biasedmatrixfactorization','cold_start_group','biasedmatrixfactorization'),score_by_group(dfvalid,'pred_biasedmatrixfactorization','cold_start_bucket','biasedmatrixfactorization'),\
score_by_group(dfvalid,'pred_biasedmatrixfactorization','flag_train','biasedmatrixfactorization')],axis=0)
result_biasedmatrixfactorization

,method,mse,rmse,mae,r2_score,Cold Start Group
0,biasedmatrixfactorization,0.762661,0.873305,0.686658,0.419384,No-cold-start
1,biasedmatrixfactorization,0.975590,0.987719,0.787813,0.184044,User-Cold-Start
2,biasedmatrixfactorization,1.004593,1.002294,0.866920,-0.205512,Item-Cold_start
3,biasedmatrixfactorization,1.161611,1.077781,1.000000,-0.161611,User-Item-cold-start
0,biasedmatrixfactorization,0.762661,0.873305,0.686658,0.419384,0
1,biasedmatrixfactorization,0.975705,0.987778,0.788064,0.184027,1
0,biasedmatrixfactorization,0.781557,0.884057,0.695652,0.402036,0


In [69]:
results_concat=pd.concat ([result_biasedmatrixfactorization,
result_svdplusplus,
result_svd,
result_userknncosine,
result_userknnpearson,
result_itemknnpearson,
result_itemknncosine,
result_itemavg,
result_useravg,
result_globalavg],axis=0)

In [70]:
pickle.dump(results_concat,open(f'{DATAPATH}/diff_results.pkl','wb'))

In [2]:
results_concat[:25]

NameError: name 'results_concat' is not defined

In [72]:
pickle.dump([dftrain,dfvalid,results_concat],open(f'{DATAPATH}/df_side_cold_runother.pkl','wb'))

In [1]:
results_concat.sort_index(by='rmse')


NameError: name 'results_concat' is not defined